LOAD DATA

Source code: https://machinelearningmastery.com/how-to-develop-an-auxiliary-classifier-gan-ac-gan-from-scratch-with-keras/

In [3]:
# Deep Learning
# Library
from keras.models import Model

# Reading CSV

import numpy as np
import pandas as pd

# MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from keras import Input, backend
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.models import Sequential, Model
import time
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from keras.optimizers import Adam, SGD
#import keras.backend.tensorflow_backend as K
from math import sqrt
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
#mine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
import keras
from keras import utils
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten

from scipy import interp
from sklearn.metrics import roc_auc_score

In [4]:
train_df = pd.read_csv('extracted_dataset.csv')

scalers = {}
train_X = None
data = []
le = preprocessing.LabelEncoder()
for col_name in ('tot_dnp3_payload_len', 'rttd','min_payload_len','dst_bytes','flag','contains_dnp3_pckt',
                 'srv_count','dst_host_count','func_code_not_supported_count','dst_host_same_srv_rate',
                 'src_bytes','count'):

#for col_name in ('path', 't','state','seq','data_cnt'):

    if train_df[col_name].dtypes == object or train_df[col_name].dtypes == bool :
        train_df[col_name] = le.fit_transform(train_df[col_name]) 
    scalers[col_name] = MinMaxScaler(feature_range=(0, 1))
    data.append(scalers[col_name].fit_transform(np.array(train_df[col_name]).reshape(-1, 1)).reshape(-1))

train_X = np.column_stack(data)
train_X_rs = train_X.reshape(train_X.shape[0],train_X.shape[1])

Y = list(train_df['label'])
for i in range(len(Y)):
    if Y[i] == 'normal':
        Y[i] = 0.0 
  #  elif Y[i] == 'dos':
     #   Y[i] = 1.0 
    else:
        Y[i] = 1.0 # convert 'dos' to 1.0, other things to 0.0
Y = np.array(Y)



#train_X, X_test, Y, y_test = train_test_split(train_X, Y, test_size = 0.20)


In [5]:
Y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [6]:
print(train_X_rs.shape)
print(Y.shape)

(861, 12)
(861,)


In [7]:
# data split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_X_rs, Y, test_size=0.3, random_state=42)
print(X_train.shape)
print(y_train.shape)
                                                    # stratify = y: means make train and test set have same distribution

(602, 12)
(602,)


GAN 1 Demention

In [29]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec  2 14:55:25 2019

@author: ncelik34
"""

# -*- coding: utf-8 -*-
"""
Created on Wed Nov 27 11:07:16 2019

@author: ncelik34
"""
from sklearn.preprocessing import MinMaxScaler
# example of fitting an auxiliary classifier gan (ac-gan) on fashion mnsit
from numpy import zeros
from numpy import ones
import numpy as np
import pandas as pd
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.initializers import RandomNormal
from matplotlib import pyplot
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

# define the standalone discriminator model
def define_discriminator(in_shape=(12,1), n_classes=2):
    # image input
    in_image = Input(shape=in_shape)
    
    #down sample to
    fe = Conv1D(16, 3, strides=2, padding='same')(in_image)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.2)(fe)
    
    #down sample to
    fe = Conv1D(32, 3, strides=2, padding='same')(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.2)(fe)
    
    #down sample to
    fe = Conv1D(64, 3, strides=2, padding='same')(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.2)(fe)
    
    #downsample one more
    fe = Conv1D(128, 3, strides=2, padding='same')(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.2)(fe)
   
    # flatten feature maps
    fe = Flatten()(fe)
    
    # real/fake output
    out1 = Dense(1, activation='sigmoid')(fe) # idenmtify r/f
    
    # class label output
    out2 = Dense(n_classes, activation='softmax')(fe) # classify 
    
    # define model
    model = Model(in_image, [out1, out2])
    
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    model.summary()
    return model



# define the standalone generator model
def define_generator(latent_dim, n_classes=2):
    depth = 32 #32
    ks = 3
    dropout = 0.25
    dim = 96 #

    # label input
    in_label = Input(shape=(1,))
    
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)# ?
    
    # linear multiplication
    n_nodes = 96 * 1
    li = Dense(n_nodes)(li)
    
    # reshape to additional channel
    li = Reshape((96, 1, 1))(li)
    
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    
    # foundation for 
    n_nodes = dim*depth
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((dim, 1, depth))(gen)
    
    # merge image gen and label input
    merge = Concatenate()([gen, li]) #gen=96,1,32 x li=96,1,1
    # upsample to 
    gen = Conv2DTranspose(16, 3, strides=(2,1), padding='same')(merge)
    gen = BatchNormalization()(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    
    #upsample to
    gen = Conv2DTranspose(8, 3, strides=(2,1), padding='same')(gen)
    gen = BatchNormalization()(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((12,-1))(gen)
    
    # upsample to 
    gen = Conv1D(1, 3, strides=1, padding='same')(gen)
    out_layer = Activation('tanh')(gen)
    
    # define model
    model = Model([in_lat, in_label], out_layer)
    model.summary()
    return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect the outputs of the generator to the inputs of the discriminator
    gan_output = d_model(g_model.output)
    # define gan model as taking noise and label and outputting real/fake and label outputs
    model = Model(g_model.input, gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model
 
    
# load images
def load_real_samples():
    (trainX, trainy) = (X_train, y_train)
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    print('Shape of training dataset: ', 'X_train: ',X.shape, 'y_train: ',trainy.shape)
    return [X, trainy]


# select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y


# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=2):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples) #check these labels!
    return [z_input, labels]


# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [images, labels_input], y
 
    
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, d_model, latent_dim, dataset, n_samples=50):
    # prepare fake examples
    [X, nmn_label], nmn_y = generate_fake_samples(g_model, latent_dim, n_samples) #TODO!:Numan (nmns were _ and _) - change labels in this row and debug!
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    
    # evaluate the classifier model
    X, y = dataset
    _, acc1, acc2 = d_model.evaluate(X, y, verbose=0)
    print('Classifier Accuracy 1: %.3f%%' % (acc1 * 100))
   # filename1 = 'g_model_SGAN_%04d.h5' % (step+1)
    #g_model.save(filename1)
   # print('Classifier Accuracy 2: %.3f%%' % (acc2 * 100)) 
#     for i in range(100):
#         np.savetxt('test_raw_nc%d.csv' % (step), X[:,:,0], delimiter=',')
#         np.savetxt('test_cat_nc%d.csv' % (step), nmn_label[:],delimiter=',')
#         filename2 = 'model_%04d.h5' % (step+1)
#         g_model.save(filename2)


# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=30, n_batch=64):
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    print('batch per epoch: %d' % bat_per_epo)
    
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print('number of steps: %d' % n_steps)
    
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    
    # manually enumerate epochs
    for i in range(n_steps):
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
        
        # update discriminator model weights
        _,d_r1,d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
        
        # generate 'fake' examples
        [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        
        # update discriminator model weights
        _,d_f,d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
        
        # prepare points in latent space as input for the generator
        [z_input, z_labels] = generate_latent_points(latent_dim, n_batch)
        
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        
        # update the generator via the discriminator's error
        _,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
        
        # summarize loss on this batch
        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_r1,d_r2*100, d_f,d_f2, g_1,g_2))
        
        # evaluate the model performance every 'epoch'
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, d_model, latent_dim, dataset)

In [30]:
#============================================Train model---------
# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

Model: "functional_67"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           [(None, 12, 1)]      0                                            
__________________________________________________________________________________________________
conv1d_55 (Conv1D)              (None, 6, 16)        64          input_34[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_77 (LeakyReLU)      (None, 6, 16)        0           conv1d_55[0][0]                  
__________________________________________________________________________________________________
dropout_44 (Dropout)            (None, 6, 16)        0           leaky_re_lu_77[0][0]             
______________________________________________________________________________________

>1, dr[0.944,126.878], df[0.734,0.851], g[0.702,0.694]
>2, dr[0.874,123.465], df[0.750,0.925], g[0.692,0.698]
>3, dr[0.873,113.601], df[0.693,0.760], g[0.688,0.690]
>4, dr[0.885,82.952], df[0.808,1.063], g[0.687,0.686]
>5, dr[0.778,116.285], df[0.755,1.087], g[0.684,0.695]
>6, dr[0.799,90.728], df[0.742,0.774], g[0.684,0.693]
>7, dr[0.947,59.658], df[0.741,0.860], g[0.678,0.696]
>8, dr[0.920,78.381], df[0.709,0.980], g[0.675,0.697]
>9, dr[0.821,71.647], df[0.772,1.017], g[0.672,0.689]
Classifier Accuracy 1: 69.721%
>10, dr[0.701,66.612], df[0.720,0.996], g[0.669,0.689]
>11, dr[0.903,63.273], df[0.628,0.830], g[0.664,0.688]
>12, dr[0.823,47.593], df[0.740,1.140], g[0.658,0.696]
>13, dr[0.830,49.158], df[0.773,0.856], g[0.662,0.696]
>14, dr[0.977,36.341], df[0.721,0.597], g[0.656,0.691]
>15, dr[0.850,32.865], df[0.682,0.874], g[0.651,0.692]
>16, dr[0.901,47.299], df[0.729,0.800], g[0.649,0.703]
>17, dr[0.823,41.843], df[0.690,0.958], g[0.645,0.683]
>18, dr[0.869,41.905], df[0.715,0.937],

>143, dr[0.810,21.920], df[0.774,0.025], g[0.623,0.040]
>144, dr[0.809,19.096], df[0.689,0.038], g[0.690,0.028]
Classifier Accuracy 1: 86.374%
>145, dr[0.859,17.024], df[0.712,0.043], g[0.616,0.023]
>146, dr[0.691,3.865], df[0.722,0.043], g[0.630,0.029]
>147, dr[0.749,19.162], df[0.725,0.038], g[0.612,0.023]
>148, dr[0.707,3.865], df[0.670,0.030], g[0.643,0.020]
>149, dr[0.689,2.926], df[0.767,0.053], g[0.669,0.019]
>150, dr[0.715,5.238], df[0.718,0.040], g[0.619,0.017]
>151, dr[0.764,12.502], df[0.665,0.059], g[0.627,0.020]
>152, dr[0.723,3.881], df[0.712,0.026], g[0.605,0.019]
>153, dr[0.746,4.154], df[0.707,0.050], g[0.660,0.014]
Classifier Accuracy 1: 85.400%
>154, dr[0.854,14.123], df[0.776,0.023], g[0.645,0.017]
>155, dr[0.715,22.423], df[0.692,0.034], g[0.683,0.016]
>156, dr[0.740,13.627], df[0.754,0.036], g[0.591,0.016]
>157, dr[0.753,35.582], df[0.708,0.029], g[0.700,0.015]
>158, dr[0.760,2.096], df[0.713,0.025], g[0.674,0.015]
>159, dr[0.724,8.415], df[0.845,0.030], g[0.617,0

In [31]:
#==========================================Test Model=========================================

# example of loading the classifier model and generating images
from keras.models import load_model

# load the model
model = load_model('g_model_SGAN_0270.h5')
model.summary()

Model: "functional_69"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_36 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_35 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_47 (Dense)                (None, 3072)         310272      input_36[0][0]                   
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 1, 50)        100         input_35[0][0]                   
______________________________________________________________________________________